<a href="https://colab.research.google.com/github/wessware/Boosting_Algorithms/blob/main/models/catb_trial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
pip install catboost

In [28]:
import pandas as pd
from catboost import CatBoostClassifier
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import numpy as np
from sklearn.model_selection import GridSearchCV

In [51]:
data_clean = pd.read_csv('/content/sample_data/clean_data_loan_modelling.csv')
data_ordinal = pd.read_csv('/content/sample_data/loan_data_clean.csv')

In [30]:
data_clean

,Loan_Status,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Gender_nan,...,Married_Yes,Married_nan,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Self_Employed_nan,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,1,0.0,5849,0.0,146.412162,360.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0,1.0,4583,1508.0,128.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1,0.0,3000,0.0,66.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1,0.0,2583,2358.0,120.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1,0.0,6000,0.0,141.000000,360.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,1,0.0,2900,0.0,71.000000,360.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
610,1,3.0,4106,0.0,40.000000,180.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
611,1,1.0,8072,240.0,253.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
612,1,2.0,7583,0.0,187.000000,360.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [52]:
data_ordinal

,Unnamed: 0,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,0.0,0,0,376,0.0,146.412162,360.0,1.0,2,1
1,1,1,1,1.0,0,0,306,1508.0,128.000000,360.0,1.0,0,0
2,2,1,1,0.0,0,1,139,0.0,66.000000,360.0,1.0,2,1
3,3,1,1,0.0,1,0,90,2358.0,120.000000,360.0,1.0,2,1
4,4,1,0,0.0,0,0,381,0.0,141.000000,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,609,0,0,0.0,0,0,125,0.0,71.000000,360.0,1.0,0,1
610,610,1,1,3.0,0,0,275,0.0,40.000000,180.0,1.0,0,1
611,611,1,1,1.0,0,0,431,240.0,253.000000,360.0,1.0,2,1
612,612,1,1,2.0,0,0,422,0.0,187.000000,360.0,1.0,2,1


In [53]:
x = data_clean.drop(columns='Loan_Status', index=None)
y = data_clean['Loan_Status']

In [54]:
x_ord = data_ordinal.drop(columns='Loan_Status', index=None)
y_ord = data_ordinal['Loan_Status']

In [55]:
x_ord

,Unnamed: 0,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,0,1,0,0.0,0,0,376,0.0,146.412162,360.0,1.0,2
1,1,1,1,1.0,0,0,306,1508.0,128.000000,360.0,1.0,0
2,2,1,1,0.0,0,1,139,0.0,66.000000,360.0,1.0,2
3,3,1,1,0.0,1,0,90,2358.0,120.000000,360.0,1.0,2
4,4,1,0,0.0,0,0,381,0.0,141.000000,360.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
609,609,0,0,0.0,0,0,125,0.0,71.000000,360.0,1.0,0
610,610,1,1,3.0,0,0,275,0.0,40.000000,180.0,1.0,0
611,611,1,1,1.0,0,0,431,240.0,253.000000,360.0,1.0,2
612,612,1,1,2.0,0,0,422,0.0,187.000000,360.0,1.0,2


In [56]:
y_ord

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 614, dtype: int64

In [32]:
y

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 614, dtype: int64

In [33]:
x

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Gender_nan,Married_No,Married_Yes,Married_nan,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Self_Employed_nan,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.0,5849,0.0,146.412162,360.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.0,4583,1508.0,128.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,3000,0.0,66.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,2583,2358.0,120.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,6000,0.0,141.000000,360.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,2900,0.0,71.000000,360.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
610,3.0,4106,0.0,40.000000,180.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
611,1.0,8072,240.0,253.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
612,2.0,7583,0.0,187.000000,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [34]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=30)

In [57]:
x_train_ord, x_test_ord, y_train_ord, y_test_ord = train_test_split(x_ord, y_ord, test_size=0.2, random_state=17)

In [36]:
cb_loan = CatBoostClassifier(n_estimators=100,
                       loss_function='Logloss',
                       learning_rate=0.1,
                       depth=3, 
                        task_type='CPU',
                       random_state=1,
                       verbose=False)

In [58]:
cb_loan.fit(x_train_ord, y_train_ord)

In [37]:
cb_loan.fit(x_train, y_train)

In [38]:
loan_predicted = cb_loan.predict(x_test)

In [59]:
predicted_ord = cb_loan.predict(x_test_ord)

In [39]:
model_rmse = np.sqrt(mse(y_test, loan_predicted))

In [61]:
model_rmse_ord = np.sqrt(mse(y_test_ord, predicted_ord))

In [62]:
model_rmse_ord

0.4324255657551935

In [40]:
model_rmse

0.48556395842728817

In [41]:
cb_loan_grid = CatBoostClassifier(n_estimators=100,
                       loss_function='Logloss',
                       learning_rate=0.1,
                        task_type='CPU',
                       random_state=1,
                       verbose=False)

In [42]:
grid_loan = {
    'max_depth' : [3, 4, 5],
    'n_estimators' : [100, 200, 300]
}

In [43]:
grid_cat = GridSearchCV(
    estimator = cb_loan_grid,
    param_grid= grid_loan,
    scoring= 'accuracy',
    cv = 5
)

In [44]:
grid_cat.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f2d7c5d2f90>,
             param_grid={'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [63]:
grid_cat.fit(x_train_ord, y_train_ord)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f2d7c5d2f90>,
             param_grid={'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [45]:
loan_predicted = grid_cat.predict(x_test)


In [64]:
loan_pred_ord = grid_cat.predict(x_test_ord)

In [68]:
model_rmse_grid = np.sqrt(mse(y_test, loan_predicted))


In [65]:
model_rmse_ord_grid = np.sqrt(mse(y_test_ord, loan_pred_ord))

In [69]:
model_rmse_grid

0.4938647983247948

In [66]:
model_rmse_ord_grid

0.4685212856658182

In [70]:
print(model_rmse)
print(model_rmse_ord)
print(model_rmse_grid)
print(model_rmse_ord_grid)

0.4938647983247948
0.4324255657551935
0.4938647983247948
0.4685212856658182
